# Projeto MegaDados

**Alunos:**
    
+ Rafael Almada
+ Warlen Cesar Rodrigues
    
**Professor:** Fábio Ayres

In [1]:
import pyspark
from math import log10

In [2]:
sc = pyspark.SparkContext(appName='Projeto e-sports')

In [3]:
sc

<SparkContext master=local[*] appName=Projeto e-sports>

In [4]:
rdd = sc.sequenceFile('part-00000')
rdd

MapPartitionsRDD[2] at mapPartitions at SerDeUtil.scala:244

In [5]:
# result = rdd.take(10)
# result[0]

### Montando RDD para Valorant

In [6]:
#Pegar os documentos que possuem a palavra Valorant
def getValorantWord(item):
    if 'Valorant' in item[1]:
        return item
    
rdd_valorant = rdd.filter(getValorantWord)
rdd_valorant.count()

10

### Montando RDD para Counter Strike

In [8]:
#Pegar os documentos que possuem a palavra CS:GO
def getCSWord(item):
    if 'CS:GO' in item[1]:
        return item
    
rdd_cs = rdd.filter(getCSWord)
rdd_cs.count()

13

### Operacionalização para Intersecção entre Valorant e Counter Strike

In [9]:
#Pegar somente os documentos que possuem as duas palavras 
rdd_valorant_cs = rdd_valorant.intersection(rdd_cs)
rdd_valorant_cs.count()

2

#### Operacionalizando frequência de palavras (1 doc = 1 contagem)

In [10]:
def conta_palavras(item):
    url, conteudo = item
    palavras = conteudo.strip().split()
    return [(palavra, 1) for palavra in set(palavras)]

def junta_contagens(nova_contagem, contagem_atual):
    return nova_contagem + contagem_atual

rdd_valorant_cs_doc_freq = rdd_valorant_cs.flatMap(conta_palavras).reduceByKey(junta_contagens)

In [11]:
#Listando tuplas (palavra, qtde. de Docs) para intersecção de Valorant e CSGO
#Neste cenário, o valor máximo do value é a quantidade retornada por rdd_valorant_csgo.count()
result = rdd_valorant_cs_doc_freq.collect() 
sorted(result, key=lambda x : -x[1])

[('Fortnite', 2),
 ('Legends', 2),
 ('of', 2),
 ('CBLOL', 2),
 ('é', 2),
 ('o', 2),
 ('em', 2),
 ('vence', 2),
 ('semifinais', 2),
 ('Riot', 2),
 ('anuncia', 2),
 ('semanas', 2),
 ('sua', 2),
 ('novos', 2),
 ('–', 2),
 ('para', 2),
 ('chega', 2),
 ('Worlds', 2),
 ('Wild', 2),
 ('na', 2),
 ('sobre', 2),
 ('a', 2),
 ('Confira', 2),
 ('LoL:', 2),
 ('League', 2),
 ('-', 2),
 ('Seraphine', 2),
 ('atrás', 2),
 ('está', 2),
 ('e', 2),
 ('CS:GO', 2),
 ('da', 2),
 ('online', 2),
 ('Games', 2),
 ('2020', 2),
 ('2ª', 2),
 ('novo', 2),
 ('LBFF', 2),
 ('vaga', 2),
 ('Etapa', 2),
 ('do', 2),
 ('Rift', 2),
 ('com', 2),
 ('as', 2),
 ('garante', 2),
 ('skins', 2),
 ('Valorant', 2),
 ('final', 2),
 ('de', 2),
 ('2020:', 2),
 ('temporada', 1),
 ('TRABALHE', 1),
 ('Among', 1),
 ('personagens', 1),
 ('STRIKE', 1),
 ('das', 1),
 ('us', 1),
 ('G2', 1),
 ('outubro,', 1),
 ('no', 1),
 ('liderança', 1),
 ('Loops', 1),
 ('campeã', 1),
 ('Como', 1),
 ('Previsão', 1),
 ('Mundial', 1),
 ('desafios', 1),
 ('Darius,'

#### Fazendo o recorte de palavras muito específicas ou muito comuns

In [12]:
N = rdd.count()
DOC_COUNT_MIN = 2 # Isso deve mudar de amostra para amostra
DOC_COUNT_MAX = 0.7 * N # Isso deve mudar de amostra para amostra (0.7 * rdd.count())

def filtra_doc_freq(item):
    contagem = item[1]
    return (contagem < DOC_COUNT_MAX) & (contagem >= DOC_COUNT_MIN)

rdd_valorant_cs_freq_filtrado = rdd_valorant_cs_doc_freq.filter(filtra_doc_freq)

In [13]:
rdd_valorant_cs_freq_filtrado.take(10)

[('Fortnite', 2),
 ('Legends', 2),
 ('of', 2),
 ('CBLOL', 2),
 ('é', 2),
 ('o', 2),
 ('em', 2),
 ('vence', 2),
 ('semifinais', 2),
 ('Riot', 2)]

#### Calculo do IDF para as palavras encontradas em documentos que possuem Valorant E Counter Strike

In [14]:
def idf(item):
    palavra, contagem = item
    return (palavra, log10(N/contagem))

rdd_idf = rdd_valorant_cs_freq_filtrado.map(idf)
result = rdd_idf.collect()
sorted(result, key=lambda x : x[1])[-100:]

[('Fortnite', 4.256874025393585),
 ('Legends', 4.256874025393585),
 ('of', 4.256874025393585),
 ('CBLOL', 4.256874025393585),
 ('é', 4.256874025393585),
 ('o', 4.256874025393585),
 ('em', 4.256874025393585),
 ('vence', 4.256874025393585),
 ('semifinais', 4.256874025393585),
 ('Riot', 4.256874025393585),
 ('anuncia', 4.256874025393585),
 ('semanas', 4.256874025393585),
 ('sua', 4.256874025393585),
 ('novos', 4.256874025393585),
 ('–', 4.256874025393585),
 ('para', 4.256874025393585),
 ('chega', 4.256874025393585),
 ('Worlds', 4.256874025393585),
 ('Wild', 4.256874025393585),
 ('na', 4.256874025393585),
 ('sobre', 4.256874025393585),
 ('a', 4.256874025393585),
 ('Confira', 4.256874025393585),
 ('LoL:', 4.256874025393585),
 ('League', 4.256874025393585),
 ('-', 4.256874025393585),
 ('Seraphine', 4.256874025393585),
 ('atrás', 4.256874025393585),
 ('está', 4.256874025393585),
 ('e', 4.256874025393585),
 ('CS:GO', 4.256874025393585),
 ('da', 4.256874025393585),
 ('online', 4.256874025393585

#### Calculo da frequencia e relevância das palavras

In [15]:
def freq_palavra(item):
    palavra, contagem = item
    return (palavra, log10(1+contagem))

def relevancia(item):
    palavra, freq = item
    freq_calc = freq_palavra(item)
    idf_calc = idf(item)
    relev = freq_calc[1]*idf_calc[1]
    return (palavra, relev)

In [16]:
freq_filtrado_sort_by_key_rdd = rdd_valorant_cs_freq_filtrado.sortByKey()
rdd_relevancia_ambas_palavras_mesmo_doc = freq_filtrado_sort_by_key_rdd.map(relevancia)
res = rdd_relevancia_ambas_palavras_mesmo_doc.collect()
sorted(res, key=lambda x : x[1])[-100:]

[('-', 2.0310450761793275),
 ('2020', 2.0310450761793275),
 ('2020:', 2.0310450761793275),
 ('2ª', 2.0310450761793275),
 ('CBLOL', 2.0310450761793275),
 ('CS:GO', 2.0310450761793275),
 ('Confira', 2.0310450761793275),
 ('Etapa', 2.0310450761793275),
 ('Fortnite', 2.0310450761793275),
 ('Games', 2.0310450761793275),
 ('LBFF', 2.0310450761793275),
 ('League', 2.0310450761793275),
 ('Legends', 2.0310450761793275),
 ('LoL:', 2.0310450761793275),
 ('Rift', 2.0310450761793275),
 ('Riot', 2.0310450761793275),
 ('Seraphine', 2.0310450761793275),
 ('Valorant', 2.0310450761793275),
 ('Wild', 2.0310450761793275),
 ('Worlds', 2.0310450761793275),
 ('a', 2.0310450761793275),
 ('anuncia', 2.0310450761793275),
 ('as', 2.0310450761793275),
 ('atrás', 2.0310450761793275),
 ('chega', 2.0310450761793275),
 ('com', 2.0310450761793275),
 ('da', 2.0310450761793275),
 ('de', 2.0310450761793275),
 ('do', 2.0310450761793275),
 ('e', 2.0310450761793275),
 ('em', 2.0310450761793275),
 ('está', 2.0310450761793275

### Operacionalização para exclusão mútua entre Valorant e Counter Strike
### Counter Strike

#### Criando o RDD com documentos que possuem somente Counter Strike
#### Filtrando o RDD

In [17]:
rdd_cs_only = rdd_cs.subtract(rdd_valorant_cs)
rdd_cs_only_doc_freq = rdd_cs_only.flatMap(conta_palavras).reduceByKey(junta_contagens)
rdd_cs_only_doc_freq_filtrado = rdd_cs_only_doc_freq.filter(filtra_doc_freq)
rdd_cs_only_doc_freq_filtrado.take(10)

[('acusam', 5),
 ('no', 9),
 ('K62+,', 2),
 ('todas', 5),
 ('Celulares', 5),
 ('AMD', 3),
 ('Canais', 5),
 ('internet', 5),
 ('have', 2),
 ('direitos', 3)]

#### Calculando o IDF das palavras

In [18]:
rdd_cs_only_idf = rdd_cs_only_doc_freq_filtrado.map(idf)
result = rdd_cs_only_idf.collect()
sorted(result, key=lambda x : x[1])[-100:]

[('jogar', 4.256874025393585),
 ('carregamento', 4.256874025393585),
 ('num', 4.256874025393585),
 ('título', 4.256874025393585),
 ('final', 4.256874025393585),
 ('quando', 4.256874025393585),
 ('servidores', 4.256874025393585),
 ('produto', 4.256874025393585),
 ('Counter', 4.256874025393585),
 ('NetMovies:', 4.256874025393585),
 ('teaser', 4.256874025393585),
 ('placas', 4.256874025393585),
 ('produtos', 4.256874025393585),
 ('problemas', 4.256874025393585),
 ('personagens', 4.256874025393585),
 ('Puzzle', 4.256874025393585),
 ('temporada', 4.256874025393585),
 ('Twitter', 4.256874025393585),
 ('Qual', 4.256874025393585),
 ('anúncio', 4.256874025393585),
 ('/', 4.256874025393585),
 ('Se', 4.256874025393585),
 ('conta', 4.256874025393585),
 ('2,', 4.256874025393585),
 ('outro', 4.256874025393585),
 ('Shopping:', 4.256874025393585),
 ('dados', 4.256874025393585),
 ('células', 4.256874025393585),
 ('atenção', 4.256874025393585),
 ('Agora,', 4.256874025393585),
 ('bastante', 4.25687402539

#### Calculando a relevância das palavras

In [ ]:
cs_filtrado_sort_by_key_rdd = rdd_cs_only_doc_freq_filtrado.sortByKey()
rdd_relevancia_cs_only = cs_filtrado_sort_by_key_rdd.map(relevancia)
res = rdd_relevancia_cs_only.collect()
sorted(res, key=lambda x : x[1])[-100:]

### Valorant

#### Criando o RDD com documentos que possuem somente Valorant
#### Filtrando o RDD

In [ ]:
rdd_valorant_only = rdd_valorant.subtract(rdd_valorant_cs)
rdd_valorant_only_doc_freq = rdd_valorant_only.flatMap(conta_palavras).reduceByKey(junta_contagens)
rdd_valorant_only_doc_freq_filtrado = rdd_valorant_only_doc_freq.filter(filtra_doc_freq)
rdd_valorant_only_doc_freq_filtrado.take(10)

#### Calculando o IDF das palavras

In [ ]:
rdd_valorant_only_idf = rdd_valorant_only_doc_freq_filtrado.map(idf)
result = rdd_valorant_only_idf.collect()
sorted(result, key=lambda x : x[1])[-100:]

#### Calculando a relevância das palavras

In [ ]:
valorant_filtrado_sort_by_key_rdd = rdd_valorant_only_doc_freq_filtrado.sortByKey()
rdd_relevancia_valorant_only = valorant_filtrado_sort_by_key_rdd.map(relevancia)
res = rdd_relevancia_valorant_only.collect()
sorted(res, key=lambda x : x[1])[-100:]

In [ ]:
sc.stop()